In [111]:
from pymongo import MongoClient

collections = {}
client = MongoClient(host='ec2-54-153-43-230.us-west-1.compute.amazonaws.com', port=27017)

collections['isis'] = client['twitter_db']['isis']
collections['greece'] = client['twitter_db']['greece']
collections['tdf'] = client['twitter_db']['tdf']
collections['ebola'] = client['twitter_db']['ebola']
collections['immigration'] = client['twitter_db']['immigration']
collections['july4'] = client['twitter_db']['july4']


In [ ]:
collections['isis'].find_one()

In [22]:
from pymongo import ASCENDING
from pymongo import DESCENDING
for key in collections:
    try:
        collections[key].drop_index('id_str_1')
        collections[key].drop_index('created_at_1')
    except:
        continue

In [26]:
from pymongo import IndexModel
from pymongo import TEXT

index1 = IndexModel([("id_str", TEXT),("user", TEXT),("lang", TEXT)], name="text", background=True)
for key in collections:
    collections[key].create_indexes([index1])

Clean up the collections by eliminating duplicate tweet id's from the same user
This is a very cautious way to do this, a small amount at a time

In [137]:
db = client.twitter_db
pipeline = [
        { "$group" : {"_id" : {
                        "user" : "$user.id",
                        "tweet" : "$id"
                        },
                      "count" : {"$sum" : 1}}},
        { "$match" : {"count" : { "$gt" : 1}}},
        { "$sort" :  {"count" : -1}},
        { "$limit" : 10000 }
    ]
duplicates = True
while duplicates:
    docs = db.command('aggregate','isis', pipeline=pipeline, allowDiskUse = True)
    for result in docs['result']:
        objIds = db.isis.find( { "user.id" : result['_id']['user'], "id" : result['_id']['tweet']}, { "_id" : 1})
        first = True
        if objIds.count() <= 1:
            duplicates = False
            break
        for objId in objIds:
            if first:
                first = False
                continue
            else:
                db.isis.remove( { "_id" : objId['_id']})

KeyboardInterrupt: 

In [138]:
db.isis.count()

1203640

In [140]:
pipeline = [
        { "$group" : {"_id" : {
                        "user" : "$user.id",
                        "tweet" : "$id"
                        },
                      "count" : {"$sum" : 1}}},
        { "$match" : {"count" : { "$gt" : 1}}},
        { "$sort" :  {"count" : -1}},
        { "$limit" : 10 }
    ]
docs = db.command('aggregate','greece', pipeline=pipeline, allowDiskUse = True)
for doc in docs:
    print doc

ok
result


In [142]:
duplicates = True
while duplicates:
    docs = db.command('aggregate','greece', pipeline=pipeline, allowDiskUse = True)
    for result in docs['result']:
        objIds = db.greece.find( { "user.id" : result['_id']['user'], "id" : result['_id']['tweet']}, { "_id" : 1})
        first = True
        if objIds.count() <= 1:
            duplicates = False
            break
        for objId in objIds:
            if first:
                first = False
                continue
            else:
                db.greece.remove( { "_id" : objId['_id']})

KeyboardInterrupt: 